# Variable()和get_variable()的区别


## 区别之一就是Variable必须在构造的时候提供初始值，因此初始值在构造时确定。


## 区别之二就是Variable总是会创建新的变量，要实现变量共享就只能传递Python对象

In [1]:
from __future__ import print_function
from __future__ import division

import tensorflow as tf

In [2]:

tf.reset_default_graph()
x1 = tf.Variable(1, name="x")
x2 = tf.Variable(2, name="x")
print(x1)
print(x2)

# tf.Variable会自动生成名字
x3 = tf.Variable(3)
print(x3)

<tf.Variable 'x:0' shape=() dtype=int32_ref>
<tf.Variable 'x_1:0' shape=() dtype=int32_ref>
<tf.Variable 'Variable:0' shape=() dtype=int32_ref>


In [3]:
tf.reset_default_graph()
with tf.variable_scope("myscope") as scope:
    x = tf.get_variable(name="x", initializer=tf.constant([1, 2, 3]))
    print(x)
# 不加reuse=True会抛出异常，不允许创建同名的变量。
# with tf.variable_scope("myscope") as scope:
#    x = tf.get_variable(name="x", dtype=tf.int32, initializer=tf.constant([1, 2, 3]))
    
with tf.variable_scope("myscope", reuse=True) as scope:
    # 注意要加上dtype=tf.int32
    x = tf.get_variable(name="x", dtype=tf.int32, initializer=tf.constant([1, 2, 3]))
    # 我们可以不提供initializer
    # x = tf.get_variable(name="x", dtype=tf.int32)
    print(x)

<tf.Variable 'myscope/x:0' shape=(3,) dtype=int32_ref>
<tf.Variable 'myscope/x:0' shape=(3,) dtype=int32_ref>


In [4]:
tf.reset_default_graph()
with tf.variable_scope("myscope") as scope:
    x = tf.get_variable(name="x", initializer=tf.constant([1, 2, 3]))
    y = tf.get_variable(name="y1", initializer=tf.constant(1))
    print(x)
    print(y)

with tf.variable_scope("myscope") as scope:
    # y不共享
    y = tf.get_variable(name="y2", initializer=tf.constant(2))
    scope.reuse_variables()
    x = tf.get_variable(name="x", dtype=tf.int32)
    print(x)
    print(y)

<tf.Variable 'myscope/x:0' shape=(3,) dtype=int32_ref>
<tf.Variable 'myscope/y1:0' shape=() dtype=int32_ref>
<tf.Variable 'myscope/x:0' shape=(3,) dtype=int32_ref>
<tf.Variable 'myscope/y2:0' shape=() dtype=int32_ref>


## 共享Dense层参数的示例

In [5]:
tf.reset_default_graph()

x1 = tf.placeholder(dtype=tf.float32, shape=[None, 3], name="x1")
x2 = tf.placeholder(dtype=tf.float32, shape=[None, 3], name="x2")
with tf.variable_scope("myscope") as scope:
    l1 = tf.layers.Dense(units=2)
    h11 = l1(x1)
with tf.variable_scope("myscope", reuse=True) as scope:
    l2 = tf.layers.Dense(units=2)
    h12 = l2(x2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([h11, h12], feed_dict={x1: [[1, 2, 3]], x2: [[2, 4, 6]]}))


[array([[-3.8509488, -2.121961 ]], dtype=float32), array([[-7.7018976, -4.243922 ]], dtype=float32)]


## name_scope和variable_scope的区别

In [6]:
tf.reset_default_graph()
with tf.name_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

    print(v1.name)  # var1:0
    print(v2.name)  # my_scope/var2:0
    print(a.name)   # my_scope/Add:0

with tf.variable_scope("my_scope"):
    v1 = tf.get_variable("var1", [1], dtype=tf.float32)
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    a = tf.add(v1, v2)

    print(v1.name)  # my_scope/var1:0
    print(v2.name)  # my_scope_1/var2:0
    print(a.name)   # my_scope_1/Add:0

var1:0
my_scope/var2:0
my_scope/Add:0
my_scope/var1:0
my_scope_1/var2:0
my_scope_1/Add:0


In [7]:
tf.reset_default_graph()
with tf.name_scope("my_scope"):
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    print(v2.name)

with tf.name_scope("my_scope"):
    v2 = tf.Variable(1, name="var2", dtype=tf.float32)
    print(v2.name)

my_scope/var2:0
my_scope_1/var2:0
